In [1]:
from os import getcwd
from os.path import join, abspath, pardir
from sys import path

In [2]:
path

['C:\\Users\\User\\Desktop\\Courses\\SDM\\SDM-Lab-1\\src\\notebooks',
 'C:\\ProgramData\\Anaconda3\\envs\\SDM\\python39.zip',
 'C:\\ProgramData\\Anaconda3\\envs\\SDM\\DLLs',
 'C:\\ProgramData\\Anaconda3\\envs\\SDM\\lib',
 'C:\\ProgramData\\Anaconda3\\envs\\SDM',
 '',
 'C:\\ProgramData\\Anaconda3\\envs\\SDM\\lib\\site-packages',
 'C:\\ProgramData\\Anaconda3\\envs\\SDM\\lib\\site-packages\\win32',
 'C:\\ProgramData\\Anaconda3\\envs\\SDM\\lib\\site-packages\\win32\\lib',
 'C:\\ProgramData\\Anaconda3\\envs\\SDM\\lib\\site-packages\\Pythonwin',
 'C:\\ProgramData\\Anaconda3\\envs\\SDM\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\User\\.ipython']

In [3]:
parent_dir = abspath(join(getcwd(), pardir))
scripts_dir = join(parent_dir, "scripts")

In [4]:
scripts_dir

'C:\\Users\\User\\Desktop\\Courses\\SDM\\SDM-Lab-1\\src\\scripts'

In [5]:
path.append(scripts_dir)

In [6]:
from connect import Neo4jConnection

In [7]:
database='publications'

In [8]:
driver= Neo4jConnection(uri='bolt://localhost:7687', user=None, pwd=None, database=database)

In [ ]:
driver.query('''Match(n) return n''')

In [ ]:
list(r)

In [34]:
def document_unique(driver):
    driver.query('''
        CREATE CONSTRAINT document_unique IF NOT EXISTS FOR (n: Document) REQUIRE (n.doi) IS UNIQUE;
    ''')

In [35]:
def author_id_unique(driver):
    driver.query('''
        CREATE CONSTRAINT author_id_unique IF NOT EXISTS FOR (n: Author) REQUIRE (n.author_id) IS UNIQUE;
    ''')

In [36]:
def keyword_unique(driver):
    driver.query('''
        CREATE CONSTRAINT keyword_unique IF NOT EXISTS FOR (n: Keyword) REQUIRE (n.name) IS UNIQUE;
    ''')

In [37]:
def create_affiliations(driver):
    driver.query(''' LOAD CSV WITH HEADERS FROM "file:///affiliations.csv" AS x
    CREATE (n: affiliation)
    SET n = x''')

In [38]:
def create_documents(driver):
    driver.query(''' LOAD CSV WITH HEADERS FROM "file:///documents.csv" AS x
    CREATE (n: document)
    SET n = x''')

In [39]:
def create_journals(driver):
    driver.query(''' LOAD CSV WITH HEADERS FROM "file:///journals.csv" AS x
    CREATE (n: journal)
    SET n = x''')

In [40]:
def create_keywords(driver):
    driver.query(''' LOAD CSV WITH HEADERS FROM "file:///keywords.csv" AS x
    CREATE (n: keyword)
    SET n = x''')

In [48]:
def create_authors(driver):
    driver.query(''' LOAD CSV WITH HEADERS FROM "file:///authors.csv" AS x
    CREATE (n: author)
    SET n = x''')

In [42]:
def document_keyword(driver):
    driver.query('''MATCH (n: document), (m: keyword)
    WHERE m.name IN n.keywords
    CREATE (n)-[r:has]->(m)
    RETURN type(r)''')

In [43]:
def document_journal(driver):
    driver.query('''MATCH (n: document), (m: journal)
    WHERE m.name = n.source_title
    CREATE (n)-[r:published_in]->(m)
    RETURN type(r)''')

In [44]:
def document_author(driver):
    driver.query('''MATCH (n: document), (m: author)
    WHERE m.author_ids = n.author_id
    CREATE (n)-[r:written_by]->(m)
    RETURN type(r)''')

In [45]:
def author_document(driver):
    driver.query('''MATCH (n: document), (m: author)
    WHERE m.author_id = n.author_id
    CREATE (n)<-[r:author_of]-(m)
    RETURN type(r)''')

In [46]:
def author_keyword(driver):
    driver.query('''MATCH (n: author), (m: keyword)
    WHERE m.keywords IN n.keywords
    CREATE (n)-[r:has]->(m)
    RETURN type(r)''')

In [47]:
def author_affiliation(driver):
    driver.query('''MATCH (n: author), (m: affiliation)
    WHERE m.affiliations=n.affiliations
    CREATE (n)-[r:affiliated_with]->(m)
    RETURN type(r)''')

In [ ]:
MATCH (j:Journals) <-[published_in]-(d:Document)
WHERE j.name CONTAINS "Conference" 
WITH j.name AS Conference, d.title AS Paper, cited_count AS Citings
ORDER BY Conference, Paper, Citings DESC LIMIT 3
RETURN Conference, Paper
